# Entrega 2 - Clasificador (Árboles de Decisión)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue implementar un algoritmo clasificador mediante la  generación de **árboles de decisión**, el mismo se basó sobre el algoritmo ID3 sobre el cual se aplicaron una serie de modificaciones. Una de las cuales fue modificar el algoritmo con estrategias para procesar atributos continuos.


En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Entrenar y evaluar árboles de decisión sobre un determinado conjunto de datos.
- **Experiencia *E*:** Procesamiento de un conjunto de atributos y su calificación determinada.
- **Performance *P*:** Accuracy, Recall y Fall-off para cada clase de calificación.

**TODO: Performance P: Vamos a usar las 3 metricas o la "Medida-Fa"? Vamos a hacer promedios micro o macro? (ppt 8-9) Creo que usamos accuracy no como presición sino esa métrica, preguntar**

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** que permite entrenar y evaluar arboles y calificar ejemplos acorde a un árbol entrenadoo.

El objetivo del informe es centralizar la información relativa a la construcción del modelo así como los datos surgidos de las pruebas realizadas con distintas configuraciones paramétricas. Como agregado opcional, se adjuntan algunos scripts para ayudar a la lectura o probar en tiempo real ciertas funcionalidades.

Por otra parte, el programa ofrece una interfaz en consola que permite entrenar y evaluar árboles acorde a múltiples configuraciones paramétricas, mostrar los arboles entrenados y calificar elementos del conjunto asociado al árbol.

*Acá hablamos de que conviene hacer pa entender la tarea*

### 1.3. Formato
***
En las siguientes secciones se especifica el diseño del modelo, justificando la toma de decisiones a la hora de construirlo y se detallan las estrategias o algoritmos que fueron implementados para la configuración paramétrica, junto a las métricas de evaluación. Luego se detalla la metodología de experimentación y con la misma los resultados obtenidos en la configuración paramétrica y graficas sobre las métricas de los mismos. Finalmente se agregan algunas conclusiones respecto a los resultados obtenidos. 

## 2. Diseño
***
En esta sección se detallan las características del diseño utilizado para construir el modelo, se profundizan las estrategias y algoritmos empleados en la configuración paramétrica y sobre las métricas empleadas para la experimentación.

### 2.1. Preprocesamiento
***
Los datos del conjunto fueron preprocesados, estos mismos se almacenan en una lista de diccionarios, en donde cada diccionario contiene la información de los atributos de un elemento del conjunto y su clasificación. 

#### 2.1.1. Adaptación de función objetivo
***
Un paso crucial del algoritmo es decidir el mejor atributo para bifurcar el árbol. ID3 propone elegir el atributo que maximice la *Ganancia de Información*.

Sin embargo esta métrica favorece los atributos que toman múltiples valores sobre otros. A raiz de esto es que también se experimentó con otras estrategias en la configuración paramétrica. *GainRatio* y *Reducción de "impureza"*

Todas las siguientes estrategias trabajan con una métrica que determina *cuánto un atributo separa a los ejemplos según la clasificación objetivo*, esta métrica es la *Entropía* y se define como:
Sea S un conjunto de ejemplos, c una función objetivo que toma los valores c$_0$...c$_n$:
$Entropía(S) = -\sum p_i log_2(p_i)$
En donde $p_i$ es la proporción de ejemplos $x \in S \ / \ f(x)=c_i$

La entropía mide la heterogeneidad de los datos: cuanto más homogéneos, menor será la misma.

##### 2.1.1.1. Ganancia
Se define la *Ganancia de Información* de un atributo A sobre una muestra S como:

$Ganancia(S,A) = Entropía(S) - \sum_{v \in Val(A)}\frac{|S_v|}{|S|}\cdotp
 Entropía(S_v)\$

La fundamentación de tomar la ganancia es que se desea que el atributo sobre el cual se particione divida el conjunto de datos de la forma más homogénea en cuanto a su clasificación posible.

Como se mencionó anteriormente, una desventaja de la ganancia es que favorece a los atributos que asuman un espectro muy alto de valores distintos sobre otros atributos. Un conjunto amplio y uniformemente distribuido de valores para el atributo no necesariamente es una característica que debería formar parte a la hora de determinar el mejor atributo para bifurcar.

**Mencionar el ejemplo de fecha????**

##### 2.1.1.2. GainRatio
Se define:
- $SplitInformation(S,A) = - \sum_{v \in Val(A)}\frac{|S_v|}{|S|} \cdotp log_2(\frac{|S_v|}{|S|})$
- $GainRatio(S,A) = \frac{Ganancia(S,A)}{SplitInformation(S,A)}$

*SplitInformation* es sensible a que tan amplia y uniformemente el atributo distribuye sus datos, logrando así penalizar a los atributos que tomen un espectro muy amplio de valores. Una colección de *n* ejemplos completamente separados por un atributo (como una fecha) tendrá $SplitInformation = log_2(n)$. En cambio un atributo booleano que divida los mismos *n* ejemplos a la mitad tendrá $SplitInformation = 1$. Si ambos atributos A y B producen la misma ganancia de información, entonces B tendrá un GainRatio más alto.

Se observa que *SplitInformation* es la entropía de S respecto a los valores del atributo A (A diferencia de tomarla con respecto a cual valor era la clasificación)

Una desventaja de usar GainRatio en lugar de Ganancia a la hora de seleccionar atributos es que el denominador puede ser 0, lo que hace el GainRatio indefinido o muy grande para atributos que pueden tener el mismo valor para casi todos los miembros de S.

##### 2.1.1.3. Reducción de "impureza"
Se define:
- $Gini(S) = 1 - \sum p_{i}^2$
- $ImpurityReduction(S,A) = Gini(S) - \sum_{v \in Val(A)}\frac{|S_v|}{|S|} \cdotp Gini(S_v)$

*Gini impurity* mide la frecuencia con la cual un elemento aleatoriamente elegido del conjunto sería incorrectamente calificado si hubiera sido calificado aleatoriamente acorde a la distribución de calificaciones en el subconjunto. Es la suma de las probabilidades $p_i$ de un elemento siendo calificado con la califiación $i$ multiplicado por la probabilidad $\sum_{k \neq i}p_k=1 - p_i$ que es la probabilidad de un error al catagorizar el elemento. Se observa que es igual a 0 cuando en todos los casos el nodo pertenece siempre a la misma categoría.

Se observa que en un conjunto de calificaciones binarias.
$Gini(S) = 1 - \sum p_{i}^2 = 1 - p_{+}^2 - p_{-}^2$

#### 2.1.2. Interpretación de valores continuos
***
Se emplearon tres estrategias distintas para interpretar los valores de atributos continuos. Las tres tienen como fin determinar un valor para poder discretizar el intervalo continuo en dos clases, los valores menores o iguales al valor determinado y los mayores al mismo.

##### 2.1.2.1. Partir en intervalos fijos
Se determina el valor de partición como la mediana del conjunto de valores para el atributo.

La fundamentación de tomar la mediana es que es una manera intuitiva de partir el atributo de forma tal que sea distribuido parejamente en el árbol. La desventaja de tomar la mediana es que no necesariamente es el punto ideal de corte para la clasificación.

##### 2.1.2.2. Partir en intervalos variables
Con el fin de mejorar el defecto de la estrategia anterior se implementa la siguiente estrategia:
- Para cada atributo se ordena el dataset según los valores del mismo.
- Se itera sobre el dataset ordenado mirando la clasificación de cada elemnto en la lista, en el momento que esta cambia se agrega a una lista auxiliar el valor del atributo sobre el cual cambió la clasificación.
- Se determina el punto de corte con el valor de la mediana de la lista auxiliar construida.

La fundamentación de esto es mejorar la estrategia anterior, considerando la mediana de los posibles puntos de corte que realmente impactan en la clasificación.

Se observa que en el peor caso posible, en el dataset ordenado las clasificaciones alternan continuamente y partir en intervalos variables sería analogo a la estrategia anterior de partir en itervalos fijos.

##### 2.1.2.3. Partir localmente para cada nodo por el valor que maximize la ganancia
Basado en el algortimo C4.5, el algortimo sucesor de ID3, se implementa la siguiente estrategia:
- Para cada atributo se ordena el dataset según los valores del mismo.
- Se itera sobre el dataset ordenado mirando la clasificación de cada elemnto en la lista, en el momento que esta cambia se agrega a una lista auxiliar el valor del atributo sobre el cual cambió la clasificación.
- Para cada posible punto de corte encontrado se calcula la ganancia de ese punto de corte sobre el dataset.
- Se determina el punto de corte como el valor que maximiza la ganancia.

La fundamentación de esto es mejorar la estrategia anterior, dando la oportunidad de encontrar un mejor punto de corte que la mediana. Lo cual aumenta la calidad del punto de corte elegido.

Una desventaja de esto es que es de un mayor orden computacional, requiriendo una recorrida del dataset para calcular la ganancia de cada punto de corte a probar.

#### 2.1.3. Adaptación a bosque
***
ID3 tiene como resultado un árbol que califica elementos de un conjunto en todas las posibles calificaciones.

Una variante con la que se experimentó fue en lugar de generar un árbol que califique elementos en las *n* calificaciones del conjunto, generar n árboles de clasificación binaria, en donde un $árbol_i$ califica *true* si el elemento pertenece a la calificación $i$ y *false* en caso contrario.

La fundamentación para experimentar con bosques de decisión en lugar de árboles de decisiones es evaluar si el tener múltiples árboles mejora la robustez del calificador.

Es importante denotar que para entrenar bosques, el dataset debe de ser preprocesado en *n* datasets cuya calificación sea un resultado booleano.


### 2.2. Algoritmo
***
El algoritmo ID3 sigue las pautas generales del mismo visto en el teórico. Se toman en cuenta las siguientes especificaciones:

#### 2.2.1. Modelos
***
El modelo puede ser un árbol de decisión tradicional o un bosque de árboles de decisión booleanos.

**Hablar de algo más? o medio que mencionar que tanto arriba como más abajo ya se dieron más detalles de esto?, mover esta sección? matarla?**

#### 2.2.2. Algoritmo de entrenamiento
***
El algoritmo entonces puede entrenar dos tipos de modelos:
- **Arbol de decisión:** En donde hace el entrenamiento greedy estándar de ID3 y da como resultado un árbol de decisión.
- **Bosque de decisión:** En donde para cada clase del conjunto entrena un árbol booleano que califica la pertenencia de un elemento a esa clase de clasificación; da como resultado un bosque de árboles de decisión con calificación booleana.

**Aca hablamos de DECISION como algoritmo de entrenamiento, mencionamos las diferencias en el entrenamiento del arbol y del bosque**

#### 2.2.2.1. Árbol - Entrenamiento multiclase
***
Para los dos conjuntos a evaluar, la calificación es multiclase, por lo tanto el resultado del algoritmo que tabaje con un modelo de árbol de decisión es un árbol tal que califica elementos del conjunto en su clasificación original.

#### 2.2.2.2. Bosque - Entrenamiento binario
***
Un bosque es otra forma de encarar la calificación de elementos en un conjunto multiclase.

El algoritmo entrena *n* árboles, siendo *n* la cantidad de clasificaciones posibles para cada elemento. Cada uno de estos árboles representa una clase y procesa elementos dando una calificación booleana, *true* si el elemento es calificado con la clase del árbol y *false* en caso contrario.

#### 2.2.3. Algoritmo de clasificación
***
A la hora de clasificar un elemento como miembro de una clase, el procedimiento es distinto según si se está trabajando con un único árbol de decisión o un bosque de los mismos.
Aca hablamos de la clasificación en arboles de decision, mencionamos las diferencias en la clasificacion con un arbol único y con un bosque

#### 2.2.3.1. Árbol - Clasificación
***
Para un árbol de decisión se sigue el procedimiento estándar mostrado en teórico, en donde se recorre de raiz a hoja tomando decisiones en base a los valores del atributo que un nodo requiera.

El resultado final es la clasificación del elemento.

#### 2.2.3.2. Bosque - Votación
***
La clasificación de un elemento en un bosque es trivial cuando todos los árboles indican *false*, aludiendo a que el elemento no es de su clasificación y uno de los árboles indica *true*. 

En este caso el resultado final es la clasificación del elemento. Sin embargo este no es siempre el caso, dado que los árboles de decisión son sensibles al ruido.

Un problema a resolver a la hora de trabajar con bosques son los *empates*, en donde uno o más árboles devuelven *true* para determinado elemento. No es directo si el elemento *e* debe ser calificado como $i$ o $j$ cuando $árbol_i$ y $árbol_j$ indican *true*.

Para resolver este problema implementamos un sistema de "votación", en donde el desempate se solucione comparando la frecuencia de ejemplos en la muestra clasificados con las calificaciones que fueron indicadas y eligiendo el de mayor frecuencia (proporción). En caso de que haya empate se decidió elegir una clasificación aleatoria dentro de las clasificaciones con igual proporción.

**TODO: Validar esto de las proporciones**


### 2.3. Evaluación
***
A

#### 2.3.1. Métricas de evaluación
***
Aca se habla de la matriz de confusión y las metricas

#### 2.3.2. Conjuntos de evaluación
***
Los dos conjuntos sobre los cuales se trabajó fueron:
- [**Conjunto *Iris*:**](https://archive.ics.uci.edu/ml/datasets/iris) El cual clasifica plantas del genero Iris (de la tribu Irideae, perteneciente a la familia Iridaceae) según tres de sus especies.
- [**Conjunto *Covertype*:**](https://archive.ics.uci.edu/ml/datasets/Covertype) El cual clasifica terrenos de bosques según atributos cartográficos.

#### 2.3.3. Método de evaluación
***
Se experimentó con dos estrategias de evaluación.
- Validación 80/20
- Validación cruzada

Ambas estrategias subdividen el conjunto S de datos en dos conjuntos, E y T. E es usado para entrenar el modelo y T es usado para validar los datos.

**Hablar de que los subconjuntos deben de serguir la misma distribución que el original? Hablar de que estadisticamente Iris tiene un buen split y lo hace? Como es la onda con covertype??? No hablar de esto?**

##### 2.3.3.1. Validación 80/20
Para esta estratégia se agrupa en E el 80% de los datos de S. Tras entrenar el árbol se evalua con T (el otro 20% de S)

##### 2.3.3.2. Validación cruzada
Para esta estratégia se determina una cantidad de intervalos *n*. Luego S es partido en *n* intervalos.

Se entrena *n* veces, empleando el $intervalo_k$ para validar (T) y el resto de intervalos para entrenar (E). La medida se obtiene como el promedio de la medida en las *k* iteraciones.

Esta estrategia es mucho más robusta que la validación 80/20, sin embargo tiene el problema de tener un muy alto orden computacional, dado que tiene que correr *k* entrenamientos en lugar de uno.

En la experimentación se decidió fijar 10 como cantidad de intervalos.

**8? Usamos cruzada? Solo para iris?**

## 3. Experimentación
***
En esta sección se detalla el conjunto de pruebas realizadas, agregandose observaciones pertinentes y comparaciones entre modelos.

### 3.1. Metodología
***

Con el objetivo de evaluar las estrategias planteadas anteriormente y su efectividad, se probaron combinaciones de las mismas frente a los dos modelos desarrollados, se encaró la exprimentación diviendola en las siguientes etapas:

1. **Configuraciones paramétricas:** Luego de llegar a un modelo relativamente libre de errores (al menos de errores identificados), se realizaron pruebas para múltiples configuraciones paramétricas y comparar cuales de ellas resultaron ser las mejores. En esta etapa se detallan los datos obtenidos. A causa de el tamaño del conjunto Covertype, no fue posible probar todas las configuraciones paramétricas para el mismo, se realizaron sólo las que teóricamente deberían dar un mejor resultado; este no fue el caso para Iris donde se probaron y juntaron datos de todas las configuraciones paramétricas.
<br><br>
2. **Elección y comparación de representantes:** A falta de suficientes corridas para generar agrupamientos de configuraciones, se compararon manualmente los resultados obtenidos en las pruebas de cada configuración paramétrica de la etapa anterior.

### 3.2. Recopilación y ajuste
***

A

### 3.3. Configuraciones paramétricas
***

#### 3.3.1. Instancias
***
A

#### 3.3.2. Parámetros
***
A

#### 3.3.3. Resultados
***
**Capaz podemos matar esta sección y hacer sus subsecciones que sean secciones 3.3.3. y 3.3.4.**

##### 3.3.3.1. Matrices de Confusión
***
**Acá va el lennyscript de matrices**

##### 3.3.3.2. Comparación de resultados
***
**Acá va el lennyscript de graficas**


#### 3.3.3.1. Instancia 1
***
#### 3.3.3.1.1. Resultados de distintas configuraciones paramétricas
***

A
<table>
    <tr>
        <th>Iteraciones</th>
        <th>Turnos</th>
        <th>Ratio $\gamma$</th>
        <th>Pesos iniciales</th>
        <th>Pesos finales (aprox)</th>
        <th>Partidas ganadas</th>
        <th>Partidas perdidas</th>
        <th>Partidas empatadas</th>
        <th>Ratio de victorias</th>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.50, 0.00, 1.00, 0.20, 0.42]</td>
        <td>13%</td>
        <td>7%</td>
        <td>80%</td>
        <td>0,65</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.27, 0.00, 0.08, 0.91, 1.00]</td>
        <td>7%</td>
        <td>2%</td>
        <td>91%</td>
        <td>0,78</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.68, 0.24, 0.00, 1.00, 0.38]</td>
        <td>21%</td>
        <td>6%</td>
        <td>73%</td>
        <td>0,78</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 1.00, 0.76, 0.69, 0.60]</td>
        <td>35%</td>
        <td>14%</td>
        <td>51%</td>
        <td>0,71</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>58%</td>
        <td>24%</td>
        <td>18%</td>
        <td>0,70</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 0.56, 1.00, 0.36, 0.29]</td>
        <td>48%</td>
        <td>34%</td>
        <td>18%</td>
        <td>0,58</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.20, 0.68, 1.00, 0.32, 0.00]</td>
        <td>44%</td>
        <td>32%</td>
        <td>24%</td>
        <td>0,57</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.21, 0.74, 0.96, 1.00, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.39, 0.80, 1.00, 0.53, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.00, 0.39, 1.00, 0.53, 0.18]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[1.00, 0.86, 0.96, 0.36, .00]</td>
        <td>31%</td>
        <td>6%</td>
        <td>63%</td>
        <td>0,83</td>
    </tr>
    <caption>Tabla 1 - Resultados de entrenamiento de <b>Instancia 1</b> para cada configuración paramétrica</caption>
</table>

#### 3.3.3.1.2. Resultados a lo largo de las iteraciones para distintas configuraciones paramétricas
***
A

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- A

#### 3.3.3.1.3. Error Cuadrático Medio cada 10 iteraciones para distintas configuraciones paramétricas
***

A

<div style="margin-top: 16px; margin-bottom: 24px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ErrorsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ErrorAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width:24%; text-align: center;">
        <img src="img/1_ErrorsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
	<img src="img/1_ErrorAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- A

### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Métricas
***
Habiendo entrenado 24 jugadores distintos, resulta pertinente determinar de la forma menos errónea posible, cuales de ellos tuvieron un **mejor** rendimiento. Se utilizan dos métricas distintas con este fin:
- **Cantidad de Partidas Ganadas (_win rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas jugadas independientemente a su resultado, siguiendo la fórmula $$winRate = \frac{ganadas}{jugadas}$$
<br><br>
- **Ratio de Victorias (_victory rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas con algún resultado, es decir, no empatadas, siguiendo la fórmula $$victoryRate = \frac{ganadas}{ganadas + perdidas}$$
<br><br>
El motivo de tomar la segunda métrica además de la primera, es el hecho de que un jugador con más victorias que otro no es necesariamente mejor. Sea el escenario donde un **jugador 1** gana y pierde 50 partidas respectivamente y un **jugador 2** gana 25 y empata 75. Si bien **jugador 1** ganó el doble de partidas que **jugador 2**, perdió 50 partidas más. El motivo de los empates de **jugador 2** es indeterminado, por lo que no se puede decir quien de los dos es mejor, pero es importante tener en cuenta que **jugador 1 no es necesariamente mejor**.

#### 3.4.2. Elección y comparación
***
Una vez determinadas las métricas, se realizaron las siguientes gráficas para comparar utilizando la cantidad de partidas ganadas:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>

**Observaciones generales:**
- La **instancia 2** alcanzó rendimientos más altos que la **instancia 1**
- Observando la gráfica, los modelos de la **instancia 1** se encuentran en condiciones parejas, excepto por aquellos entrenados utilizando $\gamma = 0.5$.
- Observando la gráfica, los "mejores" modelos de la **instancia 2** son aquellos con un peor ratio de victorias en la tabla 2 (compensado por su cantidad de partidas ganadas). Se determina que el peso de la cantidad de partidas ganadas es mayor al del ratio de victorias en este caso, ya que los otros ejemplos presentan una gran cantidad de empates.

**Interpretaciones:**
- La **instancia 1** es más robusta en relación a las distintas configuraciones paramétricas. La **instancia 2** en cambio es más sensible, devolviendo mejores resultados solo para el vector inicial con pesos pequeños. Esto puede implicar que si se entrenase más tiempo ambas instancias, la **instancia 1** alcance un mejor rendimiento que la **instancia 2**.
- Descartando el punto anterior, se interpreta que la separación de atributos sin normalizar (**instancia 2**) tiene un mejor rendimiento. Esto puede deberse a la pérdida de información al agrupar $A_i, B_i, C_i, D_i$ en $A, B, C, D$, al hecho de haber utilizado **normalización mín-máx** en los pesos, lo cual genera que uno sea siempre 0 y otro siempre 1, o a alguna otra razón no considerada.
- Tomando el hecho de que el vector con mejores resultados en la **instancia 2** es $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, se puede interpretar que priorizar $A_1$ y $A_2$ por sobre el resto de pesos es una práctica acertada para este problema.

## 4. Conclusiones
***

#### 4.1. Respecto a los Modelos
***
A

#### 4.2. Respecto a los Parámetros
***
A

#### 4.3. Respecto a los Resultados
***
A

#### 4.4. Posibles mejoras
***
A
- A